In [1]:
!pip install patool

     |████████████████████████████████| 81kB 2.5MB/s 


In [0]:
import os
import shutil

In [0]:
shutil.rmtree('mask1')

In [0]:
path = 'drive/My Drive/dataset'

In [5]:
import patoolib
patoolib.extract_archive(path+"/face.rar")

patool: Extracting drive/My Drive/dataset/face.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/dataset/face.rar"
patool:     with cwd='./Unpack_9lez31lx'
patool: ... drive/My Drive/dataset/face.rar extracted to `face'.


'face'

In [9]:
import patoolib
patoolib.extract_archive(path+"/face.rar")

patool: Extracting drive/My Drive/dataset/face.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/dataset/face.rar"
patool:     with cwd='./Unpack_abd905q1'
patool: ... drive/My Drive/dataset/face.rar extracted to `face'.


'face'

In [0]:
os.makedirs('test')
os.makedirs('train')

In [0]:
os.makedirs('test/mask')
os.makedirs('test/face')

In [0]:
os.makedirs('train/mask')
os.makedirs('train/face')

In [21]:
k = 0
for i in os.listdir('mask'):
  src = 'mask/'+i
  dest = 'train/mask/'+i
  shutil.move(src,dest)
  k = k+1
print(k)  

1703


In [22]:
print(len(os.listdir('mask')))
print(len(os.listdir('face')))

0
0


In [0]:
print(len(os.listdir('train/mask')))
print(len(os.listdir('train/face')))

In [13]:
print(len(os.listdir('test/mask')))
print(len(os.listdir('test/face')))

500
10000


In [0]:
shutil.rmtree('face')
shutil.rmtree('mask')

In [23]:
shutil.copytree('mask','train/mask')

'train/mask'

In [0]:
os.remove('mask.rar')

In [24]:
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras import optimizers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 

Using TensorFlow backend.


In [25]:
# step 1: load data

img_width = 150
img_height = 150
train_data_dir = 'train'
valid_data_dir = 'test'

datagen = ImageDataGenerator(rescale = 1./255,
                             rotation_range=60,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              zoom_range=0.2
                             )

train_generator = datagen.flow_from_directory(directory=train_data_dir,
											   target_size=(img_width,img_height),
											   classes=['face','mask'],
											   class_mode='binary',
											   batch_size=16)

validation_generator = datagen.flow_from_directory(directory=valid_data_dir,
											   target_size=(img_width,img_height),
											   classes=['face','mask'],
											   class_mode='binary',
											   batch_size=32)

Found 82171 images belonging to 2 classes.
Found 10500 images belonging to 2 classes.


In [26]:
# step-2 : build model

model =Sequential()

model.add(Conv2D(32,(3,3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

print('model complied!!')

print('starting training....')
training = model.fit_generator(generator=train_generator, steps_per_epoch=2048 // 16,epochs=30,validation_data=validation_generator,validation_steps=832//16)

print('training finished!!')



model complied!!
starting training....
Epoch 1/30
128/128 [==============================] - 85s 666ms/step - loss: 0.1662 - accuracy: 0.9810 - val_loss: 0.0120 - val_accuracy: 0.9555
Epoch 2/30
128/128 [==============================] - 84s 653ms/step - loss: 0.1323 - accuracy: 0.9775 - val_loss: 0.0244 - val_accuracy: 0.9507
Epoch 3/30
128/128 [==============================] - 83s 647ms/step - loss: 0.1171 - accuracy: 0.9790 - val_loss: 0.0603 - val_accuracy: 0.9495
Epoch 4/30
128/128 [==============================] - 85s 660ms/step - loss: 0.0994 - accuracy: 0.9785 - val_loss: 0.6971 - val_accuracy: 0.9495
Epoch 5/30
128/128 [==============================] - 82s 644ms/step - loss: 0.0948 - accuracy: 0.9800 - val_loss: 0.0763 - val_accuracy: 0.9675
Epoch 6/30
128/128 [==============================] - 83s 648ms/step - loss: 0.0801 - accuracy: 0.9824 - val_loss: 0.1448 - val_accuracy: 0.9621
Epoch 7/30
128/128 [==============================] - 82s 638ms/step - loss: 0.0879 - accur

In [0]:
##Prediction Part
import numpy as np
from keras.preprocessing import image

img_pred = image.load_img('mask.png', target_size = (150, 150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis = 0)
rslt = model.predict(img_pred)

In [38]:
rslt

array([[0.00500206]], dtype=float32)

In [39]:
print('saving weights to simple_CNN.h5')

model.save_weights('model_weights.h5')

print('all weights saved successfully !!')
#models.load_weights('models/simple_CNN.h5')

saving weights to simple_CNN.h5
all weights saved successfully !!


In [0]:
model.save('my_model.h5')

In [0]:
from tensorflow.keras.models import load_model
mode = load_model('my_model.h5')

In [0]:
##Prediction Part
import numpy as np
from keras.preprocessing import image

img_pred = image.load_img('mask.png', target_size = (150, 150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis = 0)
rslt = mode.predict(img_pred)

In [70]:
rslt

array([[0.]], dtype=float32)

In [41]:
import tensorflow as tf

modell = tf.keras.models.load_model('my_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(modell)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

4852892